In [1]:
import mouseQPCRModelSelection 
import numpy as np
import time
import AssignGPGibbsSingleLoop
import GPflow
import numpy as np
import time
import cPickle as pickle
import assigngp_dense
from matplotlib import pyplot as plt
%matplotlib inline
import branch_kernParamGPflow as bk
import BranchingTree as bt


In [2]:
subsetSelection = 0

In [3]:
pt,Yall = mouseQPCRModelSelection.LoadMouseQPCRData(subsetSelection=subsetSelection)
Y = Yall[:,0][None].T
print 'Y'
print Y.shape
strExp = 'MouseQPCR_Exp1'
m,_ = mouseQPCRModelSelection.InitModels(pt,Y)

Loaded GPLVM data/guo_ssData.p with nrowsXncols = (437, 2).
LoadMouseQPCRData output
Y
(437, 1)
Branching kernel =====================
unnamed.branchkernelparam.kern.\u001bvariance\u001b transform:+ve prior:None
[ 1.]
unnamed.branchkernelparam.kern.\u001blengthscales\u001b transform:+ve prior:None
[ 1.]
unnamed.branchkernelparam.\u001bBv\u001b transform:(none) prior:None
[[ 1.]]
unnamed.white.\u001bvariance\u001b transform:+ve prior:None
[ 1.]
Initialise models: MAP =====================
model.kern.branchkernelparam.kern.\u001bvariance\u001b transform:+ve prior:None
[ 1.]
model.kern.branchkernelparam.kern.\u001blengthscales\u001b transform:+ve prior:None
[ 1.]
model.kern.branchkernelparam.\u001bBv\u001b transform:(none) prior:None
[[ 1.]]
model.kern.white.\u001bvariance\u001b transform:+ve prior:None [FIXED]
[  1.00000000e-06]

model.likelihood.\u001bvariance\u001b transform:+ve prior:None
[ 1.]
Initialise models: VB =====================
compiling tensorflow function...
done


In [4]:
numMAPsteps = 10
D=Y.shape[1]
print D

1


In [6]:
# Do the MAP solution and plot assignments
t0 = time.time()

Kbranch = m.kern
# reinitialise hyperparameters
Kbranch.white.variance = 1e-6
Kbranch.branchkernelparam.kern.lengthscales = 100 # 20 + (90. - b) / 2. # 65
Kbranch.branchkernelparam.kern.variance = 2 #0.0012 #  2.3158
m.likelihood.variance = 0.08

# set branching point
m.kern.branchkernelparam.Bv.fixed = True
m.kern.branchkernelparam.Bv = pt.min(-1) # identify two trajectories
# should recompute Kernel everytime we update kernel hyperparameters
m.CompileAssignmentProbability(fDebug=False,fMAP=True) 

print '============> B=' + str(m.kern.branchkernelparam.Bv._array.flatten())

# Random assignment for given branch point

np.random.seed(47)

randomAssignment = AssignGPGibbsSingleLoop.GetRandomInit(pt,m.kern.branchkernelparam.Bv._array,m.indices)
print 'MAP assignment.'
(chainState, bestAssignment,_,condProbs) = \
    m.InferenceGibbsMAP(fReturnAssignmentHistory=True,fDebug=False,\
    maximumNumberOfSteps=numMAPsteps,\
    startingAssignment=list(randomAssignment))
    

elapsedMAPTime = time.time() - t0
print 'MAP times %g secs'%elapsedMAPTime

============> B=[ 0.]
MAP assignment.
Performing MAP inference with B=[ 0.]
Converged after 2 iterations, lik=52.228
MAP times 415.387 secs


In [7]:
m

Name,values,prior,constriant
model.kern.branchkernelparam.kern.variance,[ 2.],None,+ve
model.kern.branchkernelparam.kern.lengthscales,[ 100.],None,+ve
model.kern.branchkernelparam.Bv,[[ 0.]],None,[FIXED]
model.kern.white.variance,[ 1.00000000e-06],None,[FIXED]
model.likelihood.variance,[ 0.08],None,+ve


In [8]:
# Plot MAP solution
assigngp_dense.PlotSample(D,m.XExpanded[bestAssignment, : ],3,Y,Bcrap,lw=5.,fs=30, \
                      mV=None, figsizeIn=(D*10,D*7),title='Posterior B=%.1f -loglik= %.2f VB= %.2f'%(b,0,0))

NameError: name 'Bcrap' is not defined

In [ ]:
strSaveState = 'InitialAllocationForMAPv1'
saveDict = {'XExpanded':m.XExpanded, 'bestAssignment':bestAssignment, \
            'b':m.kern.branchkernelparam.Bv._array[0], 'pt':pt, 'YGPLVM':m.Y, \
            'lenscale':m.kern.branchkernelparam.kern.lengthscales._array[0],\
            'var':m.kern.branchkernelparam.kern.variance._array[0],\
            'wvar':m.kern.white.variance._array[0],\
            'lvar':m.likelihood.variance._array[0],\
            'indices':m.indices\
           }
pickle.dump( saveDict, open( 'modelfiles/'+strSaveState + '.p', "wb" ) )
